In [222]:
import ee
import geemap


In [223]:
## authenticates the session with EE
ee.Authenticate()

# creates the connection with EE. 
ee.Initialize()

In [224]:
# Define ROI for soil
roi_soil = ee.Geometry.Rectangle([163.078070, -77.625204, 163.07800, -77.625340])

In [225]:
# Pull the image for the soil endmember
soil1 = ee.Image('LANDSAT/LC08/C02/T2_TOA/LC08_055116_20231205').select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8'])

In [226]:
def get_brightness(image):
    """
    Compute the brightness of an image by summing the visible bands (B2, B3, B4, B5, B6, B7, B8).
    """
    brightness = image.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']).reduce(ee.Reducer.sum())
    return brightness

In [227]:
def select_brightest_pixels(image, roi, n=10):
    """
    Select the n brightest pixels from the image based on brightness.
    """
    # Compute brightness for each pixel by summing the selected bands (e.g., B2, B3, B4, etc.)
    brightness = image.select(['B2', 'B3', 'B4', 'B5', 'B6']).reduce(ee.Reducer.sum())
    
    # Sample the image to get pixel values within the region of interest (ROI)
    sampled_points = brightness.sample(region=roi, scale=30, numPixels=1000)
    
    # Sort the points by brightness in descending order
    sorted_points = sampled_points.sort('sum', False)  # Sort by 'sum' of brightness
    
    # Select the top n brightest points
    top_n_brightest_points = sorted_points.limit(n)
    
    return top_n_brightest_points

In [255]:
def get_average_of_brightest(image, roi, n=10):
  """
  For the top n brightest pixels, calculate the mean value for each band.
  """
  brightest_points = select_brightest_pixels(image, roi, n).toList(n)  # Convert to list (if possible)

  # Get a list of bands
  bands = image.bandNames() 

  # Create an empty dictionary to store mean values for each band
  mean_values = {}

  for band in bands:
    # Extract the values for the current band from the brightest points
    band_values = brightest_points.select([band]) 
    # Calculate the mean of the band values
    mean_value = band_values.reduceColumns(ee.Reducer.mean(), selectors=[band]).get('mean')
    # Add the mean value to the dictionary with band name as key
    mean_values[band] = mean_value

  return ee.Dictionary(mean_values)

In [256]:
# Calculate the mean for soil endmember
soil_mean = soil1.reduceRegion(ee.Reducer.mean(), roi_soil, 30).values()
soil_mean = ee.List(soil_mean)
print(soil_mean.getInfo())


[0.20610506406852178, 0.1811014584132603, 0.17480088344642095, 0.15912956637995582, 0.17721728554793767, 0.16994245563234603, 0.8713829432215009]


In [258]:
def spectral_unmixing(image, roi, n=10):
    """
    Perform spectral unmixing using the average values of the top n brightest pixels as endmembers.
    """
    average_endmembers = get_average_of_brightest(image, roi, n) 

    # Use the dictionary directly to access band means
    endmembers = [ee.List(average_endmembers), soil_mean] 

    unmixed_image = image.unmix(endmembers, True, True) 

    return unmixed_image.set('system:time_start', image.get('system:time_start'))


In [259]:
def define_endmembers(image, roi, n=10):

  # Calculate mean values for each band of the brightest pixels
  brightest_pixel_means = get_average_of_brightest(image, roi, n)

  # Ensure soil_mean is an ee.List
  soil_mean = ee.List(soil_mean) 

  # Define endmembers as a list of lists
  endmembers = [brightest_pixel_means, soil_mean] 

  return endmembers

In [271]:
def perform_unmixing(image, endmembers):
  
  unmixed_image = image.unmix(endmembers, True, True) 

  return unmixed_image.set('system:time_start', image.get('system:time_start'))

In [272]:
# Define a mosaic-by-date function
def mosaic_by_date(imcol):
    """
    Create a mosaic for each unique date in the image collection.
    """
    imlist = imcol.toList(imcol.size())
    def get_date(image):
        return ee.Image(image).date().format("YYYY-MM-dd")

    unique_dates = imlist.map(lambda im: get_date(im)).distinct()

    def create_mosaic(date_str):
        date = ee.Date(date_str)
        mosaic = imcol.filterDate(date, date.advance(1, 'day')).mosaic()
        return mosaic.set({
            'system:time_start': date.millis(),
            'system:id': date.format('YYYY-MM-dd')
        })

    mosaic_imlist = unique_dates.map(create_mosaic)
    return ee.ImageCollection(mosaic_imlist)

In [273]:
# Define the region of interest (ROI) for the area to process
roi = ee.Geometry.Rectangle([162.277817, -77.740157, 163.272100, -77.576571])  # Define your ROI here

In [274]:
# Define start and end date for the image collection filter
start_date = "2016-03-06"
end_date = "2025-01-01"

In [275]:
# Example usage with an image collection (e.g., LANDSAT/LC08)
s2 = ee.ImageCollection('LANDSAT/LC08/C02/T2_TOA')\
    .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8'])\
    .filterDate(start_date, end_date)\
    .filterBounds(roi)\
    .sort('DATE_ACQUIRED')

# Create a mosaic by date
s3 = mosaic_by_date(s2)

In [276]:
# Clip all the images in the s3 collection down to the ROI
def clip_image(image):
    return image.clip(roi)

In [277]:
#apply clip to image collection
l8_clipped = s3.map(clip_image)

In [278]:
# Process each image in the collection
def process_images(image):
    # Apply spectral unmixing on the image
    unmixed_image = perform_unmixing(image, roi)

    # Optionally reduce across bands (if spectral unmixing adds multiple bands)
    reduced_image = unmixed_image.reduce(ee.Reducer.mean())

    # Store the date as metadata
    date = image.date().format('YYYY-MM-dd')
    unmixed_image = reduced_image.set('acquired_date', date)

    return unmixed_image

In [279]:
def process_images(image):
    endmembers = define_endmembers(image, roi) 
    unmixed_image = perform_unmixing(image, endmembers) 
    return unmixed_image

In [280]:
# Apply the process_images function to the filtered image collection l8_clipped
processed_images = l8_clipped.map(process_images)

TypeError: 'List' object is not iterable

In [281]:
# Print processed image information
first_image = processed_images.first()
print(first_image.getInfo())  # Print the first image's metadata

EEException: A mapped function's arguments cannot be used in client-side operations

In [117]:
# plot the first image of the output processed files to check output

Map = geemap.Map(zoom = 10, center = [-77.616808, 163.077952])
Map.addLayer(roi)
Map.addLayer(first_image)
Map

Map(center=[-77.616808, 163.077952], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…